# Business Analysis for Northwinds Shipping Co.

This is a mock business analysis for Northwinds Shipping Co. The project highlights knowledge of sqlite in Python and knowledge of hypothesis testing to uncover statistically significant differences in business operations.

### Business Questions to answer
- Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?

In [1]:
import pandas as pd
import numpy as np
from numpy import std, mean, sqrt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sqlite3
from sqlite3 import Error
import itertools

import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
from statsmodels.stats.power import TTestIndPower, TTestPower

#File with functions written for the project
from ProjectFunctions import get_sample, get_sample_mean, cohens_d
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
conn = sqlite3.Connection('Northwind_small.sqlite')
cur = conn.cursor()

In [3]:
table_names = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', conn)
table_names = table_names.drop([5,8,9], axis=0)

tn_list = table_names.name
select_template = 'SELECT * FROM {name}'
fd = {}
for tn in tn_list:
    query = select_template.format(name = tn)
    fd[tn] = pd.read_sql(query, conn)
    
fd["Order"] = pd.read_sql('''SELECT * FROM "Order"''', conn)

In [4]:
#Making a df for each of the tables generated from the SQL tables

Ordr = pd.DataFrame(fd["Order"])
Employee = pd.DataFrame(fd["Employee"])
Category = pd.DataFrame(fd["Category"])
Customer = pd.DataFrame(fd["Customer"])
Shipper = pd.DataFrame(fd["Shipper"])
Supplier = pd.DataFrame(fd["Supplier"])
Product = pd.DataFrame(fd["Product"])
OrderDetail = pd.DataFrame(fd["OrderDetail"])
Region = pd.DataFrame(fd["Region"])
Territory = pd.DataFrame(fd["Territory"])
EmployeeTerritory = pd.DataFrame(fd["EmployeeTerritory"])

### EDA

![title](Northwind_ERD_updated.png)

In [5]:
#While the above was provided by the company, this code provides heads of all tables
tables = [Ordr, Employee, Category, Customer, Shipper, Supplier, Product, 
          OrderDetail, Region, Territory, EmployeeTerritory]
for i in tables:
    print(i.head())

      Id CustomerId  EmployeeId   OrderDate RequiredDate ShippedDate  ShipVia  \
0  10248      VINET           5  2012-07-04   2012-08-01  2012-07-16        3   
1  10249      TOMSP           6  2012-07-05   2012-08-16  2012-07-10        1   
2  10250      HANAR           4  2012-07-08   2012-08-05  2012-07-12        2   
3  10251      VICTE           3  2012-07-08   2012-08-05  2012-07-15        1   
4  10252      SUPRD           4  2012-07-09   2012-08-06  2012-07-11        2   

   Freight                   ShipName           ShipAddress        ShipCity  \
0    32.38  Vins et alcools Chevalier    59 rue de l'Abbaye           Reims   
1    11.61         Toms Spezialitäten         Luisenstr. 48         Münster   
2    65.83              Hanari Carnes       Rua do Paço, 67  Rio de Janeiro   
3    41.34       Victuailles en stock    2, rue du Commerce            Lyon   
4    51.30           Suprêmes délices  Boulevard Tirou, 255       Charleroi   

       ShipRegion ShipPostalCode ShipC

In [6]:
OrderDetail.head()

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0
3,10249/14,10249,14,18.6,9,0.0
4,10249/51,10249,51,42.4,40,0.0


In [7]:
OrderDetail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 6 columns):
Id           2155 non-null object
OrderId      2155 non-null int64
ProductId    2155 non-null int64
UnitPrice    2155 non-null float64
Quantity     2155 non-null int64
Discount     2155 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 101.1+ KB


In [8]:
for i in OrderDetail.columns:
    print(i, len(set(OrderDetail[i])))

Id 2155
OrderId 830
ProductId 77
UnitPrice 116
Quantity 55
Discount 11


In [9]:
OrderDetail['Discount'].value_counts()

0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
0.03       3
0.02       2
0.01       1
0.04       1
0.06       1
Name: Discount, dtype: int64

In [10]:
OrderDetail = OrderDetail[(OrderDetail['Discount'] != 0.06) & (OrderDetail['Discount'] != 0.03) & (OrderDetail['Discount'] != 0.02) & (OrderDetail['Discount'] != 0.01) & (OrderDetail['Discount'] != 0.04)]
OrderDetail.Discount.value_counts()

0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
Name: Discount, dtype: int64

In [20]:
conn = sqlite3.Connection('Northwind_small.sqlite')
cur = conn.cursor()

od_query = pd.read_sql('''SELECT a.OrderId, a.ProductId, a.UnitPrice, a.Quantity, a.Discount, c.CategoryName 
                          FROM OrderDetail a LEFT JOIN Product b ON a.ProductId = b.ID LEFT JOIN Category c ON b.CategoryId = c.ID''', conn)

pd.DataFrame(od_query)

,OrderId,ProductId,UnitPrice,Quantity,Discount,CategoryName
0,10248,11,14.00,12,0.00,Dairy Products
1,10248,42,9.80,10,0.00,Grains/Cereals
2,10248,72,34.80,5,0.00,Dairy Products
3,10249,14,18.60,9,0.00,Produce
4,10249,51,42.40,40,0.00,Produce
...,...,...,...,...,...,...
2150,11077,64,33.25,2,0.03,Grains/Cereals
2151,11077,66,17.00,1,0.00,Condiments
2152,11077,73,15.00,2,0.01,Seafood
2153,11077,75,7.75,4,0.00,Beverages
